The basic example is taken from *Langtangen, Hans Petter, and Anders Logg. Solving PDEs in Python: The FEniCS
Tutorial I. Springer International Publishing, 2016.*

The example code has been extended with preCICE API calls and mixed boundary conditions to allow for a Dirichlet-Neumann
coupling of two separate heat equations.

The original source code can be found on https://github.com/hplgit/fenics-tutorial/blob/master/pub/python/vol1/ft03_heat.py.

Heat equation with Dirichlet conditions. (Dirichlet problem, $f_N = \mathcal{D}\left(u_C\right)$)

\begin{align*}
\frac{\partial u}{\partial t} &= \Delta u + f && \text{on the unit square $\Omega = \left[0,1\right] \times \left[0,1\right]$}\\
u &= u_C             && \text{on the coupling boundary $\Gamma_C$ at $x = 1$}\\
u &= u_D             && \text{on the remaining boundary $\Gamma = \partial \Omega \setminus \Gamma_C$}\\
u &= u_0             && \text{on $\Omega$ at t = 0}\\
u &= 1 + x^2 + \alpha y^2 + \beta t \\
f &= \beta - 2 - 2\alpha \\
\end{align*}

Heat equation with mixed boundary conditions. (Neumann problem, $u_C = \mathcal{N}\left(f_N\right)$)

\begin{align*}
\frac{\partial u}{\partial t} &= \Delta u + f && \text{on the shifted unit square $\Omega = \left[1,2\right] \times \left[0,1\right]$}\\
\frac{\text{d}u}{\text{d}\vec{n}} &= f_N             && \text{on the coupling boundary $\Gamma_C$ at $x = 1$}\\
u &= u_D             && \text{on the remaining boundary $\Gamma = \partial \Omega \setminus \Gamma_C$}\\
u &= u_0             && \text{on $\Omega$ at t = 0}\\
u &= 1 + x^2 + \alpha y^2 + \beta t \\
f &= \beta - 2 - 2\alpha \\
\end{align*}

In [ ]:
from __future__ import print_function, division
from fenics import Function, FunctionSpace, Expression, Constant, DirichletBC, TrialFunction, TestFunction, \
    File, solve, lhs, rhs, grad, inner, dot, dx, ds, interpolate, VectorFunctionSpace, MeshFunction, MPI
from fenics import SubDomain, Point, RectangleMesh, near, Function, Expression
from fenicsprecice import Adapter
from errorcomputation import compute_errors
import numpy as np
import dolfin
from dolfin import FacetNormal, dot
from enum import Enum


class ProblemType(Enum):
    """
    Enum defines problem type. Details see above.
    """
    DIRICHLET = 1  # Dirichlet problem
    NEUMANN = 2  # Neumann problem


class DomainPart(Enum):
    """
    Enum defines which part of the domain [x_left, x_right] x [y_bottom, y_top] we compute.
    """
    LEFT = 1  # left part of domain in simple interface case
    RIGHT = 2  # right part of domain in simple interface case

In [ ]:
def determine_gradient(V_g, u, flux):
    """
    compute flux following http://hplgit.github.io/INF5620/doc/pub/fenics_tutorial1.1/tu2.html#tut-poisson-gradu
    :param V_g: Vector function space
    :param u: solution where gradient is to be determined
    :param flux: returns calculated flux into this value
    """

    w = TrialFunction(V_g)
    v = TestFunction(V_g)

    a = inner(w, v) * dx
    L = inner(grad(u), v) * dx
    solve(a == L, flux)

In [ ]:
fenics_dt = .1  # time step size
# Error is bounded by coupling accuracy. In theory we would obtain the analytical solution.
error_tol = 10**-6
alpha = 3  # parameter alpha
beta = 1.3  # parameter beta

Pick Dirichlet participant $\mathcal{D}$ or Neumann participant $\mathcal{N}$ below:

In [ ]:
problem = ProblemType.DIRICHLET
#problem = ProblemType.NEUMANN

In [ ]:
if problem is ProblemType.DIRICHLET:
    domain_part = DomainPart.LEFT
elif problem is ProblemType.NEUMANN:
    domain_part = DomainPart.RIGHT

In [ ]:
y_bottom, y_top = 0, 1
x_left, x_right = 0, 2
x_coupling = 1.0  # x coordinate of coupling interface

if domain_part is DomainPart.LEFT:
    p0 = Point(x_left, y_bottom)
    p1 = Point(x_coupling, y_top)
elif domain_part is DomainPart.RIGHT:
    p0 = Point(x_coupling, y_bottom)
    p1 = Point(x_right, y_top)

class ExcludeStraightBoundary(SubDomain):
    def get_user_input_args(self, args):
        self._interface = args.interface

    def inside(self, x, on_boundary):
        tol = 1E-14
        if on_boundary and not near(x[0], x_coupling, tol) or near(x[1], y_top, tol) or near(x[1], y_bottom, tol):
            return True
        else:
            return False

class StraightBoundary(SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        if on_boundary and near(x[0], x_coupling, tol):
            return True
        else:
            return False

nx = ny = 9
mesh = RectangleMesh(p0, p1, nx, ny, diagonal="left")
coupling_boundary = StraightBoundary()
remaining_boundary = ExcludeStraightBoundary()

In [ ]:
# Define function space using mesh
V = FunctionSpace(mesh, 'P', 2)
V_g = VectorFunctionSpace(mesh, 'P', 1)
W = V_g.sub(0).collapse()

# Define boundary conditions
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t', degree=2, alpha=alpha, beta=beta, t=0)
u_D_function = interpolate(u_D, V)

if problem is ProblemType.DIRICHLET:
    # Define flux in x direction
    f_N = Expression("2 * x[0]", degree=1, alpha=alpha, t=0)
    f_N_function = interpolate(f_N, W)
    
# Define initial value
u_n = interpolate(u_D, V)
u_n.rename("Temperature", "")

In [ ]:
precice, precice_dt, initial_data = None, 0.0, None

# Initialize the adapter according to the specific participant
if problem is ProblemType.DIRICHLET:
    precice = Adapter(adapter_config_filename="precice-adapter-config-D.json")
    precice_dt = precice.initialize(coupling_boundary, read_function_space=V, write_object=f_N_function)
elif problem is ProblemType.NEUMANN:
    precice = Adapter(adapter_config_filename="precice-adapter-config-N.json")
    precice_dt = precice.initialize(coupling_boundary, read_function_space=W, write_object=u_D_function)

In [ ]:
dt = Constant(0)
dt.assign(np.min([fenics_dt, precice_dt]))

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression('beta - 2 - 2*alpha', degree=2, alpha=alpha, beta=beta, t=0)
F = u * v / dt * dx + dot(grad(u), grad(v)) * dx - (u_n / dt + f) * v * dx

bcs = [DirichletBC(V, u_D, remaining_boundary)]

In [ ]:
# Set boundary conditions at coupling interface once wrt to the coupling
# expression
coupling_expression = precice.create_coupling_expression()

In [ ]:
if problem is ProblemType.DIRICHLET:
    # modify Dirichlet boundary condition on coupling interface
    bcs.append(DirichletBC(V, coupling_expression, coupling_boundary))
if problem is ProblemType.NEUMANN:
    # modify Neumann boundary condition on coupling interface, modify weak
    # form correspondingly
    F += v * coupling_expression * dolfin.ds

In [ ]:
a, L = lhs(F), rhs(F)

# Time-stepping
u_np1 = Function(V)
u_np1.rename("Temperature", "")
t = 0

# reference solution at t=0
u_ref = interpolate(u_D, V)
u_ref.rename("reference", " ")

# mark mesh w.r.t ranks
mesh_rank = MeshFunction("size_t", mesh, mesh.topology().dim())
if problem is ProblemType.NEUMANN:
    mesh_rank.set_all(MPI.rank(MPI.comm_world) + 4)
else:
    mesh_rank.set_all(MPI.rank(MPI.comm_world) + 0)
mesh_rank.rename("myRank", "")

# Generating output files
temperature_out = File("output/%s.pvd" % precice.get_participant_name())
ref_out = File("output/ref%s.pvd" % precice.get_participant_name())
error_out = File("output/error%s.pvd" % precice.get_participant_name())
ranks = File("output/ranks%s.pvd" % precice.get_participant_name())

# output solution and reference solution at t=0, n=0
n = 0
print('output u^%d and u_ref^%d' % (n, n))
temperature_out << u_n
ref_out << u_ref
ranks << mesh_rank

error_total, error_pointwise = compute_errors(u_n, u_ref, V)
error_out << error_pointwise

# set t_1 = t_0 + dt, this gives u_D^1
# call dt(0) to evaluate FEniCS Constant. Todo: is there a better way?
u_D.t = t + dt(0)
f.t = t + dt(0)

if problem is ProblemType.DIRICHLET:
    flux = Function(V_g)
    flux.rename("Heat-Flux", "")

In [ ]:
while precice.is_coupling_ongoing():

    # write checkpoint
    if precice.is_action_required(precice.action_write_iteration_checkpoint()):
        precice.store_checkpoint(u_n, t, n)

    read_data = precice.read_data()

    # Update the coupling expression with the new read data
    precice.update_coupling_expression(coupling_expression, read_data)

    dt.assign(np.min([fenics_dt, precice_dt]))

    # Compute solution u^n+1, use bcs u_D^n+1, u^n and coupling bcs
    solve(a == L, u_np1, bcs)

    # Write data to preCICE according to which problem is being solved
    if problem is ProblemType.DIRICHLET:
        # Dirichlet problem reads temperature and writes flux on boundary to Neumann problem
        determine_gradient(V_g, u_np1, flux)
        flux_x = interpolate(flux.sub(0), W)
        precice.write_data(flux_x)
    elif problem is ProblemType.NEUMANN:
        # Neumann problem reads flux and writes temperature on boundary to Dirichlet problem
        precice.write_data(u_np1)

    precice_dt = precice.advance(dt(0))

    # roll back to checkpoint
    if precice.is_action_required(precice.action_read_iteration_checkpoint()):
        u_cp, t_cp, n_cp = precice.retrieve_checkpoint()
        u_n.assign(u_cp)
        t = t_cp
        n = n_cp
    else:  # update solution
        u_n.assign(u_np1)
        t += float(dt)
        n += 1

    if precice.is_time_window_complete():
        u_ref = interpolate(u_D, V)
        u_ref.rename("reference", " ")
        error, error_pointwise = compute_errors(u_n, u_ref, V, total_error_tol=error_tol)
        print('n = %d, t = %.2f: L2 error on domain = %.3g' % (n, t, error))
        # output solution and reference solution at t_n+1
        print('output u^%d and u_ref^%d' % (n, n))
        temperature_out << u_n
        ref_out << u_ref
        error_out << error_pointwise

    # Update Dirichlet BC
    u_D.t = t + float(dt)
    f.t = t + float(dt)

# Hold plot
precice.finalize()